***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

# **ETAPA 4 - PROCESSAMENTO PARA CAMADA GOLD**

---
---

<br>

Essa etapa será responsável por mover os dados para a camada gold, agrupando os dados e deixando pronto para uso.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
import uuid
import gc
import urllib.request
from os import listdir
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, TimestampType, LongType, DateType, TimestampType, BooleanType, DoubleType 

### Parte 2 - **Otimizar a Sessão com configurações Personalizadas**

Aqui o será configurado algumas propriedades para que o desempenho da sessão seja mais otimizado 
- Define tamanho fixo de partições para o shuffle para melhorar o paralelismo (usar ***número de partições = número de núclos de CPU * 2 ou 3*** para encontrar melhor cenário possível)
- Define o tamanho máximo de partições para evitar muitos arquivos pequenos
- Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita
- Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados
- Habilita a extensão do Spark SQL para o Delta Lake, permitindo o uso de recursos avançados como ACID transactions, schema enforcement e time travel
- Define o catálogo padrão do Spark como o catálogo Delta, garantindo que operações de leitura, escrita e gerenciamento de tabelas utilizem o engine do Delta Lake


In [0]:
spark = (
    SparkSession.builder
        .appName("Carga Delta")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "134217728") 
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
)

### Parte 3 - **Definindo Origens, Arquivos e Destinos**

`Complete as variáveis:`<br>



In [0]:
nome_datalakehouse = "dataexperts"

nome_camada_silver = "silver"
nome_volume_silver= "vendas" 

nome_camada_gold = "gold"
nome_volume_gold = "vendas" 

O código a seguir armazena em variáveis os caminhos já prontos de origem e de destino dos dados:

In [0]:
origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_silver}/{nome_volume_silver}"
destino_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_gold}/{nome_volume_gold}"
destino_tabelas = f"{nome_datalakehouse}.{nome_camada_gold}"

O código a seguir cria o volume de destino caso ele ainda não exista:

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada_gold}.{nome_volume_gold}")

### Parte 4 - **Agrupando dados**

O código a seguir faz a leitura dos arquivos da camada silver:

In [0]:
def read(volume, file_name):
    try:
        df = spark.read\
            .format("delta")\
            .load(f"{origem_dados}/{file_name}")
        return df
    except Exception as e:
        print(f"Erro ao tentar ler {file_name}: {e}")
        return 0

O código a seguir faz salva as tabelas prontas na camada gold:

In [0]:
def save_table(df, new_file_name):
    try:
        if "fato" in new_file_name:
            df_data = spark.read.format("delta").load(f"{destino_dados}/dim_data")
            
            df_fato = df\
                .join(broadcast(
                    df_data.select("sk_data", "ano", "mes")),
                      "sk_data")\
            
            df_fato\
                .write\
                .format("delta")\
                .mode("overwrite")\
                .partitionBy("ano","mes")\
                .saveAsTable(f"{destino_tabelas}.{new_file_name}")
            
            df_fato\
                .write\
                .format("delta")\
                .mode("overwrite")\
                .save(f"{destino_dados}/{new_file_name}")
            
            return df_fato
        else:
            df.write\
                .format("delta")\
                .mode("overwrite")\
                .saveAsTable(f"{destino_tabelas}.{new_file_name}")
            df.write\
                .format("delta")\
                .mode("overwrite")\
                .save(f"{destino_dados}/{new_file_name}")
            return df
    except Exception as e:
        print(f"Ocorreu um erro na função save(): {e}")
        return 0

### Parte 5 - **Utilizando as funções**

In [0]:
new_files_names = [
    "dim_categoria_produto",
    "dim_cliente",
    "dim_data", 
    "dim_localidade",
    "dim_produto",
    "fato_vendas"
] 

files = listdir(f"{origem_dados}")

for i, (file, new_file_name) in enumerate(zip(files,new_files_names)): 

    df_read = read(nome_volume_gold, file)
    df_read = save_table(df_read, new_file_name)

    print(f"Arquivo {file} processado com sucesso.")

### Parte 6 - **Limpeza de Cache e Outros**

O código a seguir libera memória de objetos não mais utilizados:

In [0]:
del files
del new_files_names
del volume
del camada

gc.collect()


<br>

---
---

### **Resultados**

Código simples para mostrar se deu certo ou não essa etapa:

In [0]:
# %skip
new_files_names = [
    "dim_categoria_produto",
    "dim_cliente",
    "dim_data", 
    "dim_localidade",
    "dim_produto",
    "fato_vendas"
] 

def see_tables(new_file_names):
    for file in new_files_names:
        df = spark.read.format("delta").load(f"/Volumes/{nome_datalakehouse}/{nome_camada_gold}/{nome_volume_gold}/{file}")
        display(df)

see_tables(new_files_names)